## Git clone

Cloning this repository after you download this ipynb file.

In [1]:
import getpass, os

#token = getpass.getpass("input Token: ")
username = "qiulinfan"
owner = "qiulinfan"
repo = "graphNL2SQL"

#os.environ["GITHUB_TOKEN"] = token
#os.environ["GITHUB_USER"] = username
os.environ["GITHUB_OWNER"] = owner
os.environ["GITHUB_REPO"] = repo

!git clone https://github.com/${GITHUB_OWNER}/${GITHUB_REPO}.git
%cd graphNL2SQL/

Cloning into 'graphNL2SQL'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 290 (delta 76), reused 73 (delta 34), pack-reused 172 (from 1)
Receiving objects: 100% (290/290), 78.53 MiB | 13.62 MiB/s, done.
Resolving deltas: 100% (145/145), done.
/content/graphNL2SQL


In [2]:
!git pull

Already up to date.


## Download and preprocess data

In [3]:
!python scripts/download_data.py

NL2SQL Dataset Downloader

[1/2] WikiSQL Dataset
----------------------------------------
WikiSQL: 100% 26.2M/26.2M [00:00<00:00, 301MB/s]
Extracting WikiSQL...
/content/graphNL2SQL/scripts/download_data.py:54: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(DATA_DIR)
WikiSQL extracted to /content/graphNL2SQL/data/wikisql

[2/2] Spider Dataset
----------------------------------------
Downloading...
From (original): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J
From (redirected): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J&confirm=t&uuid=562d7cbc-e9e2-438c-8a20-daee2fe65453
To: /content/graphNL2SQL/data/spider_data.zip
100% 206M/206M [00:02<00:00, 80.8MB/s]
Extracting Spider dataset...
Renamed spider_data/ -> spider/
Spider extracted to /content/graphNL2SQL/data/spider

Dataset Verification
WikiSQL train: 56

In [4]:
#!python scripts/prepare_training_data.py --wikisql-balanced 2000 --spider
!python scripts/prepare_training_data.py \
    --spider \
    --style structured \
    --schema-linking \
    --prompt-style detailed \
    --wikisql-balanced 5000

Processing WikiSQL train split...
Sampling 5000 balanced WikiSQL examples from train...
  Pattern distribution in dataset:
    where_only: 40598
    count: 5114
    min: 3231
    max: 3161
    avg: 2201
    sum: 2042
    select_only: 8
  Sampled 4293 examples
WikiSQL:   0% 0/4293 [00:00<?, ?it/s]2025-12-13 13:17:18.859969: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-13 13:17:18.877152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765631838.898716    3260 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765631838.9

In [5]:
import pandas as pd
pd.set_option("display.max_rows", None)   # 不省略行
pd.set_option("display.max_columns", None)  # 不省略列
pd.set_option("display.max_colwidth", None) # 不省略内容长度
df = pd.read_json("training_data/spider_train.jsonl", lines=True)
df['input'][0:3]

,input
0,"Given the following database schema and question, generate the SQL query that answers the question.\n \nGuidelines:\n- Read the schema carefully to understand available tables and columns\n- Identify which tables and columns are relevant to the question\n- Determine if JOINs are needed (when accessing columns from multiple tables)\n- Translate question constraints into WHERE conditions\n- Apply aggregations (COUNT, SUM, etc.) when the question asks for counts or totals\n- Use ORDER BY for sorting requests\n- Use LIMIT for ""top N"" or ""first N"" requests\n[SCHEMA]\n[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SCHEMA LINKS]\nQuestion word ""department"" → table: department (confidence: 1.00)\nQuestion word ""head"" → table: head (confidence: 1.00)\n\n[QUESTION]\nHow many heads of the departments are older than 56 ?"
1,"Given the following database schema and question, generate the SQL query that answers the question.\n \nGuidelines:\n- Read the schema carefully to understand available tables and columns\n- Identify which tables and columns are relevant to the question\n- Determine if JOINs are needed (when accessing columns from multiple tables)\n- Translate question constraints into WHERE conditions\n- Apply aggregations (COUNT, SUM, etc.) when the question asks for counts or totals\n- Use ORDER BY for sorting requests\n- Use LIMIT for ""top N"" or ""first N"" requests\n[SCHEMA]\n[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SCHEMA LINKS]\nQuestion word ""department"" → table: department (confidence: 1.00)\nQuestion word ""head"" → table: head (confidence: 1.00)\nQuestion word ""name"" → column: department.Name (confidence: 1.00)\nQuestion word ""age"" → column: head.age (confidence: 1.00)\nQuestion word ""born"" → column: head.born_state (confidence: 0.70)\nQuestion word ""state"" → column: head.born_state (confidence: 0.70)\n\n[QUESTION]\nList the name, born state and age of the heads of departments ordered by age."
2,"Given the following database schema and question, generate the SQL query that answers the question.\n \nGuidelines:\n- Read the schema carefully to understand available tables and columns\n- Identify which tables and columns are relevant to the question\n- Determine if JOINs are needed (when accessing columns from multiple tables)\n- Translate question constraints into WHERE conditions\n- Apply aggregations (COUNT, SUM, etc.) when the question asks for counts or totals\n- Use ORDER BY for sorting requests\n- Use LIMIT for ""top N"" or ""first N"" requests\n[SCHEMA]\n[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SCHEMA LINKS]\nQuestion word ""department"" → table: department (confidence: 1.00)\nQuestion word ""name"" → column: department.Name (confidence: 1.00)\nQuestion word ""creation"" → column: department.Creation (confidence: 1.00)\nQuestion word ""budget"" → column: department.Budget_in_Billions (confidence: 0.70)\n\n[QUESTION]\nList the creation year, name and budget of each department."


In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None)   # 不省略行
pd.set_option("display.max_columns", None)  # 不省略列
pd.set_option("display.max_colwidth", None) # 不省略内容长度
df = pd.read_json("training_data/wikisql_train.jsonl", lines=True)
df['input'][0:3]

,input
0,"Given the following database schema and question, generate the SQL query that answers the question.\n \nGuidelines:\n- Read the schema carefully to understand available tables and columns\n- Identify which tables and columns are relevant to the question\n- Determine if JOINs are needed (when accessing columns from multiple tables)\n- Translate question constraints into WHERE conditions\n- Apply aggregations (COUNT, SUM, etc.) when the question asks for counts or totals\n- Use ORDER BY for sorting requests\n- Use LIMIT for ""top N"" or ""first N"" requests\n[SCHEMA]\n[TABLES]\nManila:\n District (PK)\n s Barangay\n Population (2010 census)\n Area ( has .)\n Pop. density (per km2)\n\n[SCHEMA LINKS]\nQuestion word ""barangay"" → table: Manila (confidence: 0.73)\nQuestion word ""area"" → column: Manila.Area ( has .) (confidence: 0.70)\n\n[QUESTION]\nWhat is the total of Barangay with an area larger than 865.13?"
1,"Given the following database schema and question, generate the SQL query that answers the question.\n \nGuidelines:\n- Read the schema carefully to understand available tables and columns\n- Identify which tables and columns are relevant to the question\n- Determine if JOINs are needed (when accessing columns from multiple tables)\n- Translate question constraints into WHERE conditions\n- Apply aggregations (COUNT, SUM, etc.) when the question asks for counts or totals\n- Use ORDER BY for sorting requests\n- Use LIMIT for ""top N"" or ""first N"" requests\n[SCHEMA]\n[TABLES]\nList_of_rural_municipalities_in_Saskatchewan:\n Rural municipality (RM) (PK)\n RM No.\n SARM Div. No.\n Census Div. No.\n Population (2011)\n Population (2006)\n Change (%)\n Land area (km²)\n Population density (per km²)\n\n[SCHEMA LINKS]\nQuestion word ""population"" → column: List_of_rural_municipalities_in_Saskatchewan.Population (2011) (confidence: 0.70)\nQuestion word ""change"" → column: List_of_rural_municipalities_in_Saskatchewan.Change (%) (confidence: 0.70)\nQuestion word ""land"" → column: List_of_rural_municipalities_in_Saskatchewan.Land area (km²) (confidence: 0.70)\nQuestion word ""area"" → column: List_of_rural_municipalities_in_Saskatchewan.Land area (km²) (confidence: 0.70)\nQuestion word ""density"" → column: List_of_rural_municipalities_in_Saskatchewan.Population density (per km²) (confidence: 0.70)\n\n[QUESTION]\nWhat is the average change to have a land area of 546.74 and a population density greater than 0.5?"
2,"Given the following database schema and question, generate the SQL query that answers the question.\n \nGuidelines:\n- Read the schema carefully to understand available tables and columns\n- Identify which tables and columns are relevant to the question\n- Determine if JOINs are needed (when accessing columns from multiple tables)\n- Translate question constraints into WHERE conditions\n- Apply aggregations (COUNT, SUM, etc.) when the question asks for counts or totals\n- Use ORDER BY for sorting requests\n- Use LIMIT for ""top N"" or ""first N"" requests\n[SCHEMA]\n[TABLES]\nKangxi_radical:\n Radical (variants) (PK)\n Stroke count\n Pīnyīn\n Hiragana - Romaji\n Meaning\n Frequency\n Examples\n\n[SCHEMA LINKS]\nQuestion word ""radical"" → table: Kangxi_radical (confidence: 0.80)\nQuestion word ""STROKE COUNT"" → column: Kangxi_radical.Stroke count (confidence: 1.00)\nQuestion word ""FREQUENCY"" → column: Kangxi_radical.Frequency (confidence: 1.00)\n\n[QUESTION]\nWHAT IS THE STROKE COUNT WITH RADICAL 皿 FREQUENCY SMALLER THAN 129?"


## Training

### Install Dependencies

In [6]:
!pip install transformers datasets peft accelerate bitsandbytes torch wandb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.5 MB/s eta 0:00:00


In [7]:
!pip install -r requirements_train.txt

In [ ]:
import transformers
import peft
import accelerate

print(f"transformers: {transformers.__version__}")
print(f"peft: {peft.__version__}")
print(f"accelerate: {accelerate.__version__}")

transformers: 4.57.3
peft: 0.18.0
accelerate: 1.12.0


In [8]:
import os
import json
from pathlib import Path
from datetime import datetime

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
)

import wandb

### GPU Check

In [ ]:
import torch

print("=" * 60)
print("GPU INFORMATION")
print("=" * 60)

if torch.cuda.is_available():
    print(f"✓ CUDA available: True")
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    total_mem = props.total_memory / 1e9
    print(f"  Memory: {total_mem:.2f} GB")
    print(f"  CUDA Version: {torch.version.cuda}")

    # Recommend config based on VRAM
    if total_mem < 12:
        print(f"\n  Recommended config: small_gpu (4-bit quantization)")
    elif total_mem < 24:
        print(f"\n Recommended config: default")
    else:
        print(f"\n Recommended config: large_gpu (more LoRA capacity)")
else:
    print("✗ CUDA not available - Training will be VERY slow on CPU")

print("=" * 60)

GPU INFORMATION
✓ CUDA available: True
  GPU: NVIDIA A100-SXM4-40GB
  Memory: 42.47 GB
  CUDA Version: 12.6

 Recommended config: large_gpu (more LoRA capacity)


### Configuration

Define hyperparameters and LoRA configuration. (No need to run this cell if config.json already good.)

In [9]:
import json
import sys
sys.path.insert(0, '.')  # Add project root to path

# =============================================================================
# CONFIGURATION - Edit values below, then run this cell to save
# =============================================================================

CONFIG = {
    "model": {
        "model_name": "Qwen/Qwen2.5-7B-Instruct",  # or "microsoft/Phi-3.5-mini-instruct"
        "load_in_4bit": False,   # Recommended for <16GB VRAM
        "load_in_8bit": False,
    },
    "lora": {
    "r": 16,
    "alpha": 32,
    "dropout": 0.15,
    "target_modules": [
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj"
    ]
    },
    "training": {
        "wikisql_epochs": 0,
        "spider_epochs": 2,
        "batch_size": 1,
        "gradient_accumulation": 16,
        "learning_rate": 0.00005,
        "lr_scheduler": "cosine",
        "warmup_ratio": 0.1,
        "weight_decay": 0.05,
        "max_seq_length": 2048,
        "gradient_checkpointing": True,
        "use_bf16": True,
        "use_fp16": False,
    },
    "data": {
        "data_dir": "./training_data",
        "output_dir": "./checkpoints",
        "max_train_samples": None,  # None = use all
    },
    "wandb": {
        "enabled": True,
        "project": "nl2sql-finetuning",
        "run_name": None,  # Auto-generated if None
    },
    "save": {
        "strategy": "epoch",
        "total_limit": 5,
    },
}

# Save configuration to JSON file
with open('config.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)

print("Configuration saved to config.json")
print(f"  Model: {CONFIG['model']['model_name']}")
print(f"  LoRA rank: {CONFIG['lora']['r']}, alpha: {CONFIG['lora']['alpha']}")
print(f"  Batch size: {CONFIG['training']['batch_size']} x {CONFIG['training']['gradient_accumulation']} = {CONFIG['training']['batch_size'] * CONFIG['training']['gradient_accumulation']} effective")
print(f"  Training: {CONFIG['training']['wikisql_epochs']} epoch WikiSQL + {CONFIG['training']['spider_epochs']} epochs Spider")

Configuration saved to config.json
  Model: Qwen/Qwen2.5-7B-Instruct
  LoRA rank: 16, alpha: 32
  Batch size: 1 x 16 = 16 effective
  Training: 0 epoch WikiSQL + 2 epochs Spider


### Load Datasets & Run Training

Using encapsulated functions from `training_utils.py`

In [ ]:
from scripts.training_utils import (
    TrainingConfig,
    load_config_from_json,
    save_config_to_json,
    print_config,
    load_datasets,
    show_sample_examples,
    load_model_and_tokenizer,
    setup_lora,
    print_model_parameters,
    init_wandb,
    train_phase1_wikisql,
    train_phase2_spider,
    finish_training,
)
print("Training utilities imported!")

Training utilities imported!


In [ ]:
# Load configuration from JSON file
config = load_config_from_json('config.json')
print_config(config)

CONFIGURATION

Model: Qwen/Qwen2.5-7B-Instruct
  4-bit: False, 8-bit: False

LoRA:
  Rank: 16, Alpha: 32, Dropout: 0.15
  Target modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']

Training:
  Batch size: 1 x 16 = 16 effective
  Learning rate: 5e-05, Scheduler: cosine, Weight decay: 0.01
  Epochs: 0 WikiSQL + 2 Spider

Paths:
  Data: ./training_data
  Output: ./checkpoints

WandB: Enabled
  Project: nl2sql-finetuning


In [ ]:
# Load datasets
wikisql_train, wikisql_dev, spider_train, spider_dev = load_datasets(config.data_dir)
show_sample_examples(wikisql_train, spider_train)

DATASET INFORMATION

 WikiSQL Dataset:
   Train samples: 4,293
   Dev samples:   999
   Columns: ['input', 'output', 'schema', 'question', 'sql', 'dataset', 'table_id', 'split']

 Spider Dataset:
   Train samples: 7,000
   Dev samples:   1,034
   Columns: ['input', 'output', 'schema', 'question', 'sql', 'dataset', 'db_id', 'split', 'num_tables', 'has_join']
   Multi-table: 7,000 (100.0%)
   With JOIN:   2,783 (39.8%)

SAMPLE EXAMPLES

 WikiSQL Example:
Question: What is the total of Barangay with an area larger than 865.13?
SQL: SELECT SUM("s Barangay") FROM "table" WHERE "Area ( has .)" > 865.13

Schema preview:
[TABLES]
Manila:
    District (PK)
    s Barangay
    Population (2010 census)
    Area ( has .)
    Pop. density (per km2)

------------------------------------------------------------

 Spider Example:
Question: How many heads of the departments are older than 56 ?
SQL: SELECT count(*) FROM head WHERE age  >  56
Database: department_management

Schema preview:
[DATABASE]
dep

In [ ]:
# Load model and setup LoRA
model, tokenizer = load_model_and_tokenizer(config)
model = setup_lora(model, config)
print_model_parameters(model)

LOADING MODEL

Model: Qwen/Qwen2.5-7B-Instruct

Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]


 Model loaded!
CONFIGURING LoRA

LoRA Configuration:
   Rank (r):        16
   Alpha:           32
   Dropout:         0.15
   Target modules:  ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']

 LoRA adapters added!
MODEL PARAMETERS (AFTER LoRA)
trainable params: 40,370,176 || all params: 7,655,986,688 || trainable%: 0.5273

 Detailed Parameter Count:
   Total parameters:       7,655,986,688 (7.66B)
   Trainable parameters:      40,370,176 (40.37M)
   Frozen parameters:      7,615,616,512 (7.62B)
   Trainable %:                  0.5273%

 LoRA Adapter Size:
   LoRA parameters:           40,370,176 (40.37M)


In [ ]:
# Initialize WandB
run_name = init_wandb(config)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rynnefan (rynnefan-university-of-michigan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


 WandB: Logging in...
  (Will use default API key if no input within 5 seconds)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


  Using default API key
 WandB: Logged in successfully


Exception in thread Thread-7 (get_input):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/content/graphNL2SQL/graphNL2SQL/graphNL2SQL/scripts/training_utils.py", line 658, in get_input
    key = input("Enter WandB API key (or press Enter for default): ").strip()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'dict' object has no attribute 'strip'


 WandB initialized! Run: nl2sql_20251213_023640


### Phase 1: WikiSQL Warmup

In [ ]:
# Phase 1: WikiSQL Warmup Training
train_phase1_wikisql(model, tokenizer, wikisql_train, wikisql_dev, config, run_name)

Skipping Phase 1 (WikiSQL not available or epochs=0)


### Phase 2: Spider Main Training

In [ ]:
# Phase 2: Spider Main Training
trainer = train_phase2_spider(model, tokenizer, spider_train, spider_dev, config, run_name)

PHASE 2: Spider Main Training

Preparing Spider data...


Tokenizing Spider train:   0%|          | 0/7000 [00:00<?, ? examples/s]

Tokenizing Spider eval:   0%|          | 0/500 [00:00<?, ? examples/s]

/content/graphNL2SQL/graphNL2SQL/graphNL2SQL/scripts/training_utils.py:1003: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.



 Spider Dataset Ready:
   Train: 7,000 samples
   Eval:  500 samples

 Training Steps Configuration:
   Steps per epoch:     437
   Eval every:          43 steps (~0.1 epoch)
   Save every:          258 steps (~0.59 epoch, 6x eval)

 Starting Phase 2 training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
43,1.074100,0.905205
86,0.427100,0.421660
129,0.317400,0.399701
172,0.199600,0.465073
215,0.144200,0.536872
258,0.098000,0.595104
301,0.089700,0.576786
344,0.088100,0.603202
387,0.083100,0.614986
430,0.084100,0.645833



 Saving Phase 2 model to ./checkpoints/phase2_spider/final

 Phase 2 complete!


### Finish Training

In [ ]:
# Save the final model
# Use output directory from config
OUTPUT_DIR = config.output_dir + "/phase2_spider/final"

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")

# Finish WandB run
import wandb
if wandb.run:
    wandb.finish()
    print("WandB run finished. Check your dashboard for loss graphs!")

Model saved to ./checkpoints/phase2_spider/final


eval/loss,█▁▁▂▃▄▃▄▄▄▅▅▅▅▅▅▅▅▅▅
eval/runtime,▂▂▁▄▂▂▄▂▁▄▂▅▃▂█▃▂▁▂▇
eval/samples_per_second,▇▇█▅▇▇▅▇█▅▇▅▆▇▁▆▇█▇▃
eval/steps_per_second,▇▇█▅▇▇▅▇█▅▇▅▆▇▁▆▇█▇▃
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇█
train/grad_norm,█▃▁▁▁▁▂▂▂▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▂▃▆█████▇▇▇▇▇▇▆▆▆▅▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
train/loss,█▇▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,0.71634
eval/runtime,121.6616


WandB run finished. Check your dashboard for loss graphs!


### Download to local

In [ ]:
import shutil
DOWNLOAD_PATH = "./checkpoints/phase2_spider/checkpoint-438"

def zip_and_download(model_path):
    model_path = Path(model_path)
    if not model_path.exists():
        print(f" Path not found: {model_path}")
        return

    zip_name = f"{model_path.name}_lora_adapter"
    zip_path = f"/tmp/{zip_name}"

    print(f" Creating zip: {model_path}")
    shutil.make_archive(zip_path, 'zip', model_path.parent, model_path.name)

    zip_file = f"{zip_path}.zip"
    print(f"   Size: {os.path.getsize(zip_file)/(1024*1024):.1f} MB")

    try:
        from google.colab import files
        print(" Downloading...")
        files.download(zip_file)
    except ImportError:
        print(f"\n Download manually: {zip_file}")

zip_and_download(DOWNLOAD_PATH)

 Path not found: checkpoints/phase2_spider/checkpoint-438


## Note: Before testing, put current config file into all checkpoint directories, and then put the checkpoints folders into drive.

## Testing (Drive Loaded)

Using encapsulated functions from `testing_utils.py`

In [ ]:
from scripts.testing_utils import (
    generate_sql,
    load_finetuned_model,
    # Evaluation functions
    evaluate_model,              # Exact Match only
    evaluate_with_execution,     # Exact Match + Execution Match
    show_evaluation_examples,
    # Testing functions
    run_quick_test,
    run_interactive_test,
    test_with_university_schema,
    test_with_ecommerce_schema,
    UNIVERSITY_SCHEMA,
    ECOMMERCE_SCHEMA,
    SAMPLE_QUESTIONS,
    # EGD functions
    generate_sql_with_egd,
    evaluate_with_egd,
    comprehensive_evaluation
)
print("Testing utilities imported (including Execution Match and EGD)!")

Testing utilities imported (including Execution Match and EGD)!


### Load checkpoints we want from drive

In [20]:
# =============================================================================
# Upload Checkpoint from Google Drive and Test
# =============================================================================

import sys
from pathlib import Path
import shutil
sys.path.insert(0, '.')

from scripts.testing_utils import (
    load_finetuned_model,
    evaluate_with_execution,
    load_jsonl,
)
from scripts.training_utils import load_config_from_json

# Configuration
DRIVE_CHECKPOINT_PATH = "/content/drive/MyDrive/nl2sql/checkpoint-436"
LOCAL_CHECKPOINT_PATH = "./checkpoints/phase2_spider/checkpoint-436"
EVAL_DATA_PATH = "./training_data/dev.jsonl"
MAX_SAMPLES = None  # None for all
USE_EGD = False

# Mount Google Drive
try:
    from google.colab import drive
    if not Path("/content/drive").exists():
        drive.mount('/content/drive')
except ImportError:
    pass

# Copy checkpoint from Drive
drive_path = Path(DRIVE_CHECKPOINT_PATH)
local_path = Path(LOCAL_CHECKPOINT_PATH)

if not drive_path.exists():
    raise FileNotFoundError(f"Checkpoint not found: {drive_path}")

local_path.parent.mkdir(parents=True, exist_ok=True)
if local_path.exists():
    shutil.rmtree(local_path)
shutil.copytree(drive_path, local_path)
print(f"✓ Checkpoint copied from Drive")

# Load config
# config 文件在 HECKPOINT_PATH 目录下
try:
    config = load_config_from_json(local_path / 'config.json')
    base_model_name = config.model_name
    load_in_4bit = config.load_in_4bit
    load_in_8bit = config.load_in_8bit
except:
    base_model_name = None
    load_in_4bit = False
    load_in_8bit = False

# Load model
print("Loading model...")
model, tokenizer = load_finetuned_model(
    str(local_path),
    base_model_name=base_model_name,
    load_in_4bit=load_in_4bit,
    load_in_8bit=load_in_8bit,
)

✓ Checkpoint copied from Drive
Loading model...
Loading base model: Qwen/Qwen2.5-7B-Instruct
Loading tokenizer...


The tokenizer you are loading from 'checkpoints/phase2_spider/checkpoint-436' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading base model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading LoRA adapters...
 Model loaded successfully!


In [21]:
# Run quick test with default schema/question
run_quick_test(model, tokenizer)

QUICK TEST

Question: How many students are majoring in Computer Science?

Schema:
[TABLES]
student:
    id (PK)
    name
    age
    major

Generated SQL:
SELECT count(*) FROM student WHERE major  =  'Computer Science'


"SELECT count(*) FROM student WHERE major  =  'Computer Science'"

In [ ]:
# Load evaluation data
eval_data = load_jsonl("./training_data/spider_dev.jsonl")
train_data = load_jsonl("./training_data/spider_train.jsonl")  # 用于统计训练样本数

# Configuration
USE_EGD = False          # Use Execution-Guided Decoding for better accuracy
EGD_CANDIDATES = 5       # Number of candidates for EGD

if eval_data:
    # 使用终极评估函数
    eval_results = comprehensive_evaluation(
        model, tokenizer,
        eval_data=eval_data,
        train_data=train_data,  # 可选，用于显示每个数据库的训练样本数
        max_samples=None,       # None = 评估全部
        use_egd=USE_EGD,
        egd_candidates=EGD_CANDIDATES,
        verbose=True,           # 会自动打印详细统计
    )

    # Show example predictions
    show_evaluation_examples(eval_results, num_correct=3, num_incorrect=3)
else:
    print("Spider dev data not loaded.")

# Print summary (已经在 comprehensive_evaluation 中打印了，这里可以省略)
# 如果需要额外访问结果：
# eval_results["overall"]["em_accuracy"]
# eval_results["overall"]["ex_accuracy"]
# eval_results["by_database"]
# eval_results["by_operation"]
# eval_results["by_complexity"]

Evaluating on 1034 samples (Standard mode)...
  [10/1034] Accuracy: 80.0%
  [20/1034] Accuracy: 70.0%
  [30/1034] Accuracy: 63.3%
  [40/1034] Accuracy: 50.0%
  [50/1034] Accuracy: 46.0%
  [60/1034] Accuracy: 41.7%
  [70/1034] Accuracy: 37.1%
  [80/1034] Accuracy: 37.5%
  [90/1034] Accuracy: 35.6%
  [100/1034] Accuracy: 32.0%
  [110/1034] Accuracy: 29.1%
  [120/1034] Accuracy: 26.7%
  [130/1034] Accuracy: 24.6%
  [140/1034] Accuracy: 22.9%
  [150/1034] Accuracy: 21.3%
  [160/1034] Accuracy: 20.0%
  [170/1034] Accuracy: 19.4%
  [180/1034] Accuracy: 18.9%
  [190/1034] Accuracy: 22.1%
  [200/1034] Accuracy: 24.0%
  [210/1034] Accuracy: 25.7%
  [220/1034] Accuracy: 25.0%
  [230/1034] Accuracy: 25.7%
  [240/1034] Accuracy: 26.2%
  [250/1034] Accuracy: 26.0%
  [260/1034] Accuracy: 26.2%
  [270/1034] Accuracy: 27.0%
  [280/1034] Accuracy: 28.6%
  [290/1034] Accuracy: 29.0%
  [300/1034] Accuracy: 31.0%
  [310/1034] Accuracy: 31.0%
  [320/1034] Accuracy: 32.5%
  [330/1034] Accuracy: 33.9%
  [340

### Evaluate with Execution Match (EM + EX)

Execution Match compares the actual execution results of predicted SQL vs gold SQL on a mock database. This is more forgiving than Exact Match since different SQL queries can produce the same results.


In [ ]:
# Evaluate with both Exact Match (EM) and Execution Match (EX)
# This requires DuckDB: pip install duckdb
# Uses EGD setting from above cell

if spider_dev:
    ex_results = evaluate_with_execution(
        eval_model, eval_tokenizer,
        spider_dev,
        max_samples=EVAL_MAX_SAMPLES,
        verbose=True,
        use_egd=USE_EGD,
        egd_candidates=EGD_CANDIDATES,
    )

    print("\n" + "=" * 60)
    print("SUMMARY" + (" (with EGD)" if USE_EGD else ""))
    print("=" * 60)
    print(f"Exact Match (EM):     {ex_results['exact_match_accuracy']:.2f}%")
    print(f"Execution Match (EX): {ex_results['execution_match_accuracy']:.2f}%")

    # Show how many queries EX found correct that EM missed
    ex_only = sum(1 for r in ex_results['results']
                  if r['execution_match'] and not r['exact_match'])
    if ex_only > 0:
        print(f"\nEX found {ex_only} additional correct queries beyond EM!")
else:
    print("Spider dev data not loaded.")


Evaluating 100 samples with EM + EX (Standard mode)...
------------------------------------------------------------
    [Debug] Both SQLs failed execution:
      Gold error: Execution error: Binder Error: Cannot compare values of type VARCHAR and type INTEGER_LITERAL - an e
      Pred error: Execution error: Binder Error: Cannot compare values of type VARCHAR and type INTEGER_LITERAL - an e
  [10/100] EM: 80.0%, EX: 90.0%
    [Debug] Both SQLs failed execution:
      Gold error: Execution error: Binder Error: No function matches the given name and argument types 'avg(VARCHAR)'.
      Pred error: Execution error: Catalog Error: Table with name concert_attendance does not exist!
Did you mean "con
    [Debug] Both SQLs failed execution:
      Gold error: Execution error: Binder Error: column "average" must appear in the GROUP BY clause or must be part o
      Pred error: Execution error: Binder Error: No function matches the given name and argument types 'avg(VARCHAR)'.
  [20/100] EM: 75.

### Interactive Testing

In [ ]:
# Test with university schema
test_with_university_schema(eval_model, eval_tokenizer)

INTERACTIVE TESTING

Schema:
[DATABASE]
university

[TABLES]
student:
    student_id (PK)
    name
    age
    department_id (FK)
course:
    course_id (PK)
    title
    credits
enrollment:
    enrollment_id (PK)
    student_id (FK)
    course_id (FK)
    grade

[FOREIGN KEYS]
student.department_id -> department.department_id
enrollment.student_id -> student.student_id
enrollment.course_id -> course.course_id

------------------------------------------------------------

[1] Q: How many students are there?
    SQL: SELECT count(*) FROM student

[2] Q: What are the names of students older than 20?
    SQL: SELECT name FROM student WHERE age  >  20

[3] Q: List all courses with more than 3 credits.
    SQL: SELECT * FROM course WHERE credits  >  3

[4] Q: Which students are enrolled in the Database course?
    SQL: SELECT T1.name FROM student AS T1 JOIN enrollment AS T2 ON T1.student_id  =  T2.student_id JOIN course AS T3 ON T2.course_id  =  T3.course_id WHERE T3.title  =  'Database'

[

[{'question': 'How many students are there?',
  'sql': 'SELECT count(*) FROM student'},
 {'question': 'What are the names of students older than 20?',
  'sql': 'SELECT name FROM student WHERE age  >  20'},
 {'question': 'List all courses with more than 3 credits.',
  'sql': 'SELECT * FROM course WHERE credits  >  3'},
 {'question': 'Which students are enrolled in the Database course?',
  'sql': "SELECT T1.name FROM student AS T1 JOIN enrollment AS T2 ON T1.student_id  =  T2.student_id JOIN course AS T3 ON T2.course_id  =  T3.course_id WHERE T3.title  =  'Database'"},
 {'question': 'What is the average age of students?',
  'sql': 'SELECT avg(age) FROM student'}]

In [ ]:
# Test with ecommerce schema
test_with_ecommerce_schema(eval_model, eval_tokenizer)

INTERACTIVE TESTING

Schema:
[DATABASE]
ecommerce

[TABLES]
customer:
    customer_id (PK)
    name
    email
    city
product:
    product_id (PK)
    name
    price
    category
order:
    order_id (PK)
    customer_id (FK)
    order_date
    total_amount
order_item:
    item_id (PK)
    order_id (FK)
    product_id (FK)
    quantity

[FOREIGN KEYS]
order.customer_id -> customer.customer_id
order_item.order_id -> order.order_id
order_item.product_id -> product.product_id

------------------------------------------------------------

[1] Q: How many customers are from New York?
    SQL: SELECT count(*) FROM customer WHERE city  =  'New York'

[2] Q: What is the total revenue from all orders?
    SQL: SELECT sum(total_amount) FROM order;

[3] Q: List products with price greater than 100.
    SQL: SELECT * FROM product WHERE price > 100

[4] Q: Which customers placed orders in 2024?
    SQL: SELECT DISTINCT T1.name FROM customer AS T1 JOIN order AS T2 ON T1.customer_id  =  T2.customer_i

[{'question': 'How many customers are from New York?',
  'sql': "SELECT count(*) FROM customer WHERE city  =  'New York'"},
 {'question': 'What is the total revenue from all orders?',
  'sql': 'SELECT sum(total_amount) FROM order;'},
 {'question': 'List products with price greater than 100.',
  'sql': 'SELECT * FROM product WHERE price > 100'},
 {'question': 'Which customers placed orders in 2024?',
  'sql': 'SELECT DISTINCT T1.name FROM customer AS T1 JOIN order AS T2 ON T1.customer_id  =  T2.customer_id WHERE YEAR(T2.order_date)  =  2024'},
 {'question': 'What is the most popular product by quantity sold?',
  'sql': 'SELECT T2.name FROM ORDER_ITEM AS T1 JOIN product AS T2 ON T1.product_id  =  T2.product_id GROUP BY T1.product_id ORDER BY sum(T1.quantity) DESC LIMIT 1'}]

In [ ]:
# Custom test - modify schema and questions as needed
custom_schema = """[TABLES]
employee:
    id (PK)
    name
    department
    salary"""

custom_questions = [
    "What is the average salary?",
    "List employees in the Engineering department.",
    "Who has the highest salary?",
]

run_interactive_test(eval_model, eval_tokenizer, custom_schema, custom_questions)

INTERACTIVE TESTING

Schema:
[TABLES]
employee:
    id (PK)
    name
    department
    salary

------------------------------------------------------------

[1] Q: What is the average salary?
    SQL: SELECT avg(salary) FROM employee

[2] Q: List employees in the Engineering department.
    SQL: SELECT * FROM employee WHERE department  =  'Engineering'

[3] Q: Who has the highest salary?
    SQL: SELECT name FROM employee ORDER BY salary DESC LIMIT 1


[{'question': 'What is the average salary?',
  'sql': 'SELECT avg(salary) FROM employee'},
 {'question': 'List employees in the Engineering department.',
  'sql': "SELECT * FROM employee WHERE department  =  'Engineering'"},
 {'question': 'Who has the highest salary?',
  'sql': 'SELECT name FROM employee ORDER BY salary DESC LIMIT 1'}]